In [2]:
import matplotlib.pyplot as plt

In [3]:
from astroquery.mast import Catalogs

def query_name_for_ticid(name,radius=2./3600.):
    """
    Query name for ticid
    
    INPUT:
        name - name of star
        radius - radius to search for in degrees (2./3600. would be 2 arcseconds)
        
    EXAMPLE:
        query_name_for_ticid("GJ 699")
        
    NOTES:
        Make sure only 1 ID is returned, otherwise there might be confusion which star it is 
        If there are more than 1 IDs returned, try a smaller radius (can happen for binaries)
    """
    catalog_data = Catalogs.query_object(name, radius=5./3600., catalog="TIC")
    df = catalog_data.to_pandas()
    if len(df) > 1:
        message = '**WARNING, found {} IDs, try reducing radius**'.format(len(df))
    if len(df) == 1:
        message = 'Found 1 ID'
    print("Found object {} at: RA={:0.5f}, DEC={:0.5}. Message={}".format(name,
                                                              df['ra'].values[0],
                                                              df['dec'].values[0],
                                                              message))
    return df['ID'].values


In [4]:
from importlib import reload
import os
import glob2
import pandas as pd
import numpy as np
import sys
import glob

In [5]:
gto_files = pd.read_csv('../input/20201020_hpf_gto_targets/20201020_hpf_gto_specmatch_filenames_sncut600.csv')
gto_files

,name,filename,sn18
0,NLTT_47026,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,258.016724
1,LHS_3154,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,79.397766
2,GJ_9066,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,486.186462
3,GJ_1151,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,272.230072
4,VB_10,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,161.487061
...,...,...,...
160,NLTT_39249,NaN,0.000000
161,LHS_3247,NaN,0.000000
162,GJ_445,NaN,0.000000
163,GJ_1238,NaN,0.000000


In [6]:
gto_files_nonan = gto_files.dropna().reset_index(drop=True)
gto_files_nonan
foldername_fits = '../input/20201020_hpf_gto_targets/'
foldername_csv = '../output/20201020_hpf_gto_targets/'
files_fits = []
files_csv = []
for i, f in enumerate(gto_files_nonan.filename.values):
    basename = os.path.basename(f)
    filename = '{}{}'.format(foldername_fits,basename)
    files_fits.append(filename)
    targetname = gto_files_nonan.name.values[i]
    filename_csv = '{}{}/{}_med.csv'.format(foldername_csv,targetname,targetname)
    files_csv.append(filename_csv)
gto_files_nonan['filename_fits'] = files_fits
gto_files_nonan['filename_csv'] = files_csv
gto_files_nonan

,name,filename,sn18,filename_fits,filename_csv
0,NLTT_47026,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,258.016724,../input/20201020_hpf_gto_targets/Slope-201906...,../output/20201020_hpf_gto_targets/NLTT_47026/...
1,LHS_3154,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,79.397766,../input/20201020_hpf_gto_targets/Slope-202008...,../output/20201020_hpf_gto_targets/LHS_3154/LH...
2,GJ_9066,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,486.186462,../input/20201020_hpf_gto_targets/Slope-202007...,../output/20201020_hpf_gto_targets/GJ_9066/GJ_...
3,GJ_1151,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,272.230072,../input/20201020_hpf_gto_targets/Slope-202003...,../output/20201020_hpf_gto_targets/GJ_1151/GJ_...
4,VB_10,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,161.487061,../input/20201020_hpf_gto_targets/Slope-201810...,../output/20201020_hpf_gto_targets/VB_10/VB_10...
...,...,...,...,...,...
134,2MASS_J05253566+0826271,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,57.813309,../input/20201020_hpf_gto_targets/Slope-201903...,../output/20201020_hpf_gto_targets/2MASS_J0525...
135,TYC2597-735-1,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,163.566208,../input/20201020_hpf_gto_targets/Slope-201807...,../output/20201020_hpf_gto_targets/TYC2597-735...
136,GJ_581,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,430.932190,../input/20201020_hpf_gto_targets/Slope-202002...,../output/20201020_hpf_gto_targets/GJ_581/GJ_5...
137,GJ_1138,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,226.875320,../input/20201020_hpf_gto_targets/Slope-202002...,../output/20201020_hpf_gto_targets/GJ_1138/GJ_...


In [7]:
ticid = []
for name in gto_files_nonan['name']:
    print(name)
    try:
        results = query_name_for_ticid(name)
    except Exception as e:
        print(e)
        results = np.array(['-999'])
    ticid.append(results)
    print(results)
gto_files_nonan['ticid'] = ticid
gto_files_nonan

NLTT_47026
Found object NLTT_47026 at: RA=282.07307, DEC=7.6892. Message=Found 1 ID
['92923185']
LHS_3154
Found object LHS_3154 at: RA=241.64081, DEC=40.906. Message=Found 1 ID
['24108819']
GJ_9066


<ipython-input-7-c917646d138b>:5: TableReplaceWarning: converted column 'numcont' from integer to float
  results = query_name_for_ticid(name)
<ipython-input-7-c917646d138b>:5: TableReplaceWarning: converted column 'gaiaqflag' from integer to float
  results = query_name_for_ticid(name)
<ipython-input-7-c917646d138b>:5: TableReplaceWarning: converted column 'raddflag' from integer to float
  results = query_name_for_ticid(name)


Found object GJ_9066 at: RA=30.05398, DEC=13.052. Message=**WARNING, found 2 IDs, try reducing radius**
['404715018' '471011166']
GJ_1151
Found object GJ_1151 at: RA=177.74051, DEC=48.377. Message=Found 1 ID
['11893637']
VB_10


index 0 is out of bounds for axis 0 with size 0
['-999']
GJ_251
Found object GJ_251 at: RA=103.70399, DEC=33.268. Message=Found 1 ID
['68581262']
GJ_1215
Found object GJ_1215 at: RA=259.43371, DEC=11.67. Message=**WARNING, found 2 IDs, try reducing radius**
['1308384263' '459611421']
GJ_1154
Found object GJ_1154 at: RA=183.56896, DEC=0.62399. Message=Found 1 ID
['397052407']
GJ_4037
Found object GJ_4037 at: RA=268.88921, DEC=58.408. Message=Found 1 ID
['232626766']
LHS_3241
Found object LHS_3241 at: RA=251.63139, DEC=34.582. Message=Found 1 ID
['219066411']
GJ_4071
Found object GJ_4071 at: RA=280.68741, DEC=13.905. Message=**WARNING, found 2 IDs, try reducing radius**
['230345672' '1801578772']
LSPM_J0515+5911
Found object LSPM_J0515+5911 at: RA=78.87893, DEC=59.188. Message=Found 1 ID
['87302025']
GJ_644_C
Found object GJ_644_C at: RA=253.89690, DEC=-8.3947. Message=**WARNING, found 2 IDs, try reducing radius**
['181609153' '471012515']
GJ_643
Found object GJ_643 at: RA=253.85509, DEC

Found object NLTT_371 at: RA=2.22987, DEC=49.316. Message=Found 1 ID
['201784071']
2MASS_J17023016+6145060
Found object 2MASS_J17023016+6145060 at: RA=255.62569, DEC=61.752. Message=Found 1 ID
['459977474']
LSPM_J1650+5948
Found object LSPM_J1650+5948 at: RA=252.65412, DEC=59.806. Message=Found 1 ID
['199685853']
GJ_170
Found object GJ_170 at: RA=67.60538, DEC=39.85. Message=Found 1 ID
['238865036']
NLTT_46822
Found object NLTT_46822 at: RA=279.88785, DEC=29.871. Message=Found 1 ID
['316993844']
LSPM_J1835+3259
Found object LSPM_J1835+3259 at: RA=278.90786, DEC=32.998. Message=Found 1 ID
['27858644']
NLTT_45468
Found object NLTT_45468 at: RA=266.54923, DEC=60.406. Message=**WARNING, found 2 IDs, try reducing radius**
['219807052' '1400855815']
GJ_3252
Found object GJ_3252 at: RA=57.75020, DEC=-0.87925. Message=Found 1 ID
['401945077']
GJ_3326
Found object GJ_3326 at: RA=76.06149, DEC=11.057. Message=Found 1 ID
['415508270']
LHS_2090
Found object LHS_2090 at: RA=135.09811, DEC=21.835. M

,name,filename,sn18,filename_fits,filename_csv,ticid
0,NLTT_47026,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,258.016724,../input/20201020_hpf_gto_targets/Slope-201906...,../output/20201020_hpf_gto_targets/NLTT_47026/...,[92923185]
1,LHS_3154,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,79.397766,../input/20201020_hpf_gto_targets/Slope-202008...,../output/20201020_hpf_gto_targets/LHS_3154/LH...,[24108819]
2,GJ_9066,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,486.186462,../input/20201020_hpf_gto_targets/Slope-202007...,../output/20201020_hpf_gto_targets/GJ_9066/GJ_...,"[404715018, 471011166]"
3,GJ_1151,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,272.230072,../input/20201020_hpf_gto_targets/Slope-202003...,../output/20201020_hpf_gto_targets/GJ_1151/GJ_...,[11893637]
4,VB_10,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,161.487061,../input/20201020_hpf_gto_targets/Slope-201810...,../output/20201020_hpf_gto_targets/VB_10/VB_10...,[-999]
...,...,...,...,...,...,...
134,2MASS_J05253566+0826271,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,57.813309,../input/20201020_hpf_gto_targets/Slope-201903...,../output/20201020_hpf_gto_targets/2MASS_J0525...,[-999]
135,TYC2597-735-1,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,163.566208,../input/20201020_hpf_gto_targets/Slope-201807...,../output/20201020_hpf_gto_targets/TYC2597-735...,[284856863]
136,GJ_581,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,430.932190,../input/20201020_hpf_gto_targets/Slope-202002...,../output/20201020_hpf_gto_targets/GJ_581/GJ_5...,[36853511]
137,GJ_1138,/gpfs/group/cfb12/default/HPFData/Processed/Wa...,226.875320,../input/20201020_hpf_gto_targets/Slope-202002...,../output/20201020_hpf_gto_targets/GJ_1138/GJ_...,"[900072372, 249667452]"


In [8]:
gto_files_nonan.to_csv('../output/gto_summary2.csv',index=False)

In [7]:
# Example good:
query_name_for_ticid("2MASS_J05253566+0826271")

ResolverError: Could not resolve 2MASS_J05253566+0826271 to a sky position.

In [25]:
# Example of problematic star
# We might have to look up both of these in ExoFOP to figure out which one is correct, as this is a tight binary
query_name_for_ticid("GJ 15 B",radius=1./3600.)

Found object GJ 15 B at: RA=4.60760, DEC=44.027. Message=**WARNING, found 2 IDs, try reducing radius**


/Users/gks/opt/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: TableReplaceWarning: converted column 'numcont' from integer to float
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gks/opt/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: TableReplaceWarning: converted column 'gaiaqflag' from integer to float
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gks/opt/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: TableReplaceWarning: converted column 'raddflag' from integer to float
  This is separate from the ipykernel package so we can avoid doing imports until


array(['440109718', '471011162'], dtype=object)